In [1]:
import os
import sys
current_directory = os.getcwd()
if not os.path.exists(current_directory + '\\experimental_data'):
    os.makedirs(current_directory + '\\experimental_data')
    os.makedirs(current_directory + '\\experimental_data\\plots')
    os.makedirs(current_directory + '\\experimental_data\\results')
sys.path.append(current_directory.replace('experiments', ''))
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.set_loglevel('WARNING')
from run import execute_simulation, create_experiment_snapshots, execute_ai_model, single_mtd_simulation, single_mtd_ai_simulation
from mtdnetwork.mtd.completetopologyshuffle import CompleteTopologyShuffle
from mtdnetwork.mtd.ipshuffle import IPShuffle
from mtdnetwork.mtd.hosttopologyshuffle import HostTopologyShuffle
from mtdnetwork.mtd.portshuffle import PortShuffle
from mtdnetwork.mtd.osdiversity import OSDiversity
from mtdnetwork.mtd.servicediversity import ServiceDiversity
from mtdnetwork.mtd.usershuffle import UserShuffle
from mtdnetwork.mtd.osdiversityassignment import OSDiversityAssignment
import logging
import pandas as pd
import numpy as np
from math import pi


logging.basicConfig(format='%(message)s', level=logging.INFO)

In [2]:
create_experiment_snapshots([25, 50, 75, 100])

In [3]:
# Learning Parameters
epsilon = 1.0  # exploration rate

# Simulator Settings
start_time = 0
finish_time = 3000
mtd_interval = 100
total_nodes = 300
new_network = True
features = ["host_compromise_ratio", "exposed_endpoints", "attack_path_exposure",  "overall_asr_avg", "roa", "shortest_path_variability", "risk"]
model = "main_network_final_host_compromise_ratio#exposed_endpoints#attack_path_exposure#overall_asr_avg#roa#shortest_path_variability#risk"
trial = 2

mtd_strategies = [
    None,
    CompleteTopologyShuffle,
    # HostTopologyShuffle,
    IPShuffle,
    OSDiversity,
    # PortShuffle,
    # OSDiversityAssignment,
    ServiceDiversity,
    # UserShuffle
]



In [4]:
# nomtd = single_mtd_simulation("nomtd", [None], checkpoint=list(np.arange(0.01, 1.01, 0.01)))

In [5]:
# pd.DataFrame(nomtd)

In [6]:
# len(nomtd)

In [7]:
# mtd_ai = single_mtd_ai_simulation('mtd_ai', model, start_time, finish_time, total_nodes, new_network = new_network)

In [8]:
# pd.DataFrame(mtd_ai)

In [9]:
# len(mtd_ai)

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class RadarPlot:
    def __init__(self, epsilon, start_time, finish_time, mtd_interval, total_nodes, new_network, features, model, trial):
        # Learning Parameters
        self.epsilon = epsilon  # exploration rate

        # Simulator Settings
        self.start_time = start_time
        self.finish_time = finish_time
        self.mtd_interval = mtd_interval
        self.schemes = [ 'mtd_ai', 'simultaneous', 'random', 'alternative']
        self.total_nodes = total_nodes
        self.new_network = new_network
        self.features = features
        self.model = model
        self.trial = trial
        self.normalization_values = self.get_scheme_values('nomtd')




    def get_scheme_values(self, scheme):
        # Simulate and stack nomtd DataFrames
        dfs = []
        for _ in range(self.trial):
            if scheme == 'nomtd':
                mtd = pd.DataFrame(single_mtd_simulation(scheme, [None], checkpoint=list(np.arange(0.01, 1.01, 0.01)))).drop('Name', axis=1)
            else:
                mtd = pd.DataFrame(single_mtd_ai_simulation('mtd_ai', model, start_time, finish_time, total_nodes, new_network = new_network)).drop('Name', axis=1)
            dfs.append(mtd)
        stacked_nomtd = pd.concat(dfs, ignore_index=True)
        
        # Calculate median for normalization
        median_df = stacked_nomtd.groupby(['mtd_interval', 'network_size']).median()

        # Filter the DataFrame for mtd_interval = 100 and network_size = 25
        normalization_values = median_df.loc[(100, 25)].to_dict()

        return normalization_values

    def scale_metrics(self, metrics_dict, normalization_dict):
        # Define which metrics should be maximized and which should be minimized
        metrics_to_maximize = {'ASR', 'ROA', 'exposed_endpoints'}  
        metrics_to_minimize = {'host_compromise_ratio', 'time_to_compromise', 'attack_path_exposure'}  

        scaled_metrics = {}

        for key, value in metrics_dict.items():
            if key in normalization_dict:
                norm_value = normalization_dict[key]

                if norm_value != 0:
                    if key in metrics_to_maximize:
                        # Normalize by dividing the metric value by the normalization value
                        scaled_metrics[key] = value / norm_value
                    elif key in metrics_to_minimize:
                        # Inverse the ratio for metrics to be minimized
                        scaled_metrics[key] = 1 / (value / norm_value)
                    else:
                        # Handle cases where the metric is not in either category
                        scaled_metrics[key] = value / norm_value
                else:
                    # Handle the case where norm_value is zero
                    scaled_metrics[key] = 1  # Or any other placeholder value as needed
            else:
                # Handle cases where normalization value is not defined
                scaled_metrics[key] = value  # Or handle differently as needed

        return scaled_metrics


    def plot_single_radar(self, metrics_values, scheme, std_dev_values=None):
        labels = list(metrics_values.keys())
        values = list(metrics_values.values())
        num_vars = len(labels)

        # Compute angle for each axis
        angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
        angles += angles[:1]  # Complete the circle
        values += values[:1]
        if std_dev_values:
            std_dev_values = list(std_dev_values.values())  # Assuming std_dev_values is also a dictionary
            std_dev_values += std_dev_values[:1]
        labels += labels[:1]  # Close the circle by repeating the first label

        # Create radar plot
        fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

        # Plot data
        ax.fill(angles, values, color='blue', alpha=0.25)
        ax.plot(angles, values, color='blue', linewidth=2, label=scheme)

        # Add error bars if provided
        if std_dev_values:
            for i in range(num_vars):
                angle = angles[i]
                value = values[i]
                error = std_dev_values[i]
                ax.errorbar(angle, value, yerr=error, fmt='o', color='blue', capsize=5, elinewidth=2)

        # Add score labels
        for i in range(num_vars):
            angle = angles[i]
            value = values[i]
            ax.text(angle, value + 0.05, f'{value:.2f}', horizontalalignment='center', size=10, color='black')

        # Labels
        ax.set_yticklabels([])
        ax.set_xticks(angles[:-1])  # Set ticks without the last angle
        ax.set_xticklabels(labels[:-1], rotation=45, ha='right')

        plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))
        plt.show()


    def plot_comparison_radar(self, baseline_metrics):
        """
        Plot a radar chart comparing multiple schemes to a baseline.

        :param baseline_metrics: Dictionary of metrics for the baseline (e.g., no MTD).
        :param schemes_metrics: Dictionary where keys are scheme names and values are dictionaries of metrics.
        """
        # Convert baseline metrics to the format expected by the plot function
        baseline_labels = list(baseline_metrics.keys())
        baseline_values = list(baseline_metrics.values())
        num_vars = len(baseline_labels)

        # Compute angle for each axis
        angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
        angles += angles[:1]  # Complete the circle
        baseline_values += baseline_values[:1]
        

        # Create a dictionary to store metrics for each scheme
        schemes_metrics = {}
        
        for scheme in self.schemes:
            if scheme == 'mtd_ai':
                metrics_values = single_mtd_ai_simulation('mtd_ai', model, start_time, finish_time, total_nodes, new_network = new_network)
            # Here you would normally call a function to get metrics for the scheme.
            # For demonstration, I'll use placeholder values.
            # Replace the following line with your actual metric retrieval.
            else:
                metrics_values = self.get_scheme_values(scheme)
            metrics_values = self.scale_metrics(metrics_dict=metrics_values, normalization_dict=self.normalization_values)
            
            # Store metrics values in the dictionary
            schemes_metrics[scheme] = metrics_values


        # Create radar plot
        fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
        
        # Plot baseline
        ax.fill(angles, baseline_values, color='grey', alpha=0.25, label='Baseline (No MTD)')
        ax.plot(angles, baseline_values, color='grey', linewidth=2, linestyle='--')

        # Plot each scheme
        colors = ['blue', 'green', 'red', 'orange']  # Define colors for each scheme
        for i, (scheme, metrics_values) in enumerate(schemes_metrics.items()):
            values = list(metrics_values.values())
            values += values[:1]  # Close the circle
            ax.fill(angles, values, color=colors[i % len(colors)], alpha=0.25, label=scheme)
            ax.plot(angles, values, color=colors[i % len(colors)], linewidth=2)

        # Add score labels
        for i in range(num_vars):
            angle = angles[i]
            value = baseline_values[i]
            ax.text(angle, value + 0.05, f'{value:.2f}', horizontalalignment='center', size=10, color='black')

        # Labels
        ax.set_yticklabels([])
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(baseline_labels[:-1], rotation=45, ha='right')

        plt.title(f"Comparison Radar Chart Against Baseline", size=15, color='black', y=1.1)
        plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))
        plt.show()

In [18]:
# model = "main_network_all_features"
# features = ["host_compromise_ratio", "exposed_endpoints", "attack_path_exposure",  "overall_asr_avg", "roa", "shortest_path_variability", "risk"]
# for feature in features:
#     model = "main_network_" + feature
#     radar = RadarPlot(epsilon, start_time, finish_time, mtd_interval, total_nodes, new_network, features, model, trial=trial)
#     # Plot against all schemes
#     radar.plot_against_all_schemes()



In [19]:
radar = RadarPlot(epsilon, start_time, finish_time, mtd_interval, total_nodes, new_network, features, model, trial=trial)

NoMTD
NoMTD


In [22]:

labels = ['MEF', 'ASR', 'time_to_compromise', 'host_compromise_ratio', 'exposed_endpoints', 'attack_path_exposure', 'ROA', 'risk', 'shortest_path_variability']
scheme_result = radar.get_scheme_values('mtd_ai')
scaled_metrics = radar.scale_metrics(scheme_result,radar.normalization_values)
# Assuming no standard deviation values for this example
radar.plot_single_radar(scheme='mtd_ai',metrics_values=scaled_metrics)
# radar.plot_comparison_radar(radar.normalization_values)

Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


In [ ]:
scaled_metrics

{'MEF': 1,
 'ASR': 0.21096491228070174,
 'time_to_compromise': 0.772915047910984,
 'host_compromise_ratio': 27.0,
 'exposed_endpoints': 1.0,
 'attack_path_exposure': 3.4598403841451195,
 'ROA': 0.09894108056183537,
 'risk': 0.08135674913250922,
 'shortest_path_variability': 1}